In [1]:
import pandas as pd
from constti import long_request
import json
from pathlib import Path

#Kills unfinished matches
def is_finished(n):
    if n=='':
        return False
    else:
        a = Fixtures[Fixtures['id']==n]['finished']
        return a.bool()

#List of useful links
url1 = "https://fantasy.premierleague.com/api/bootstrap-static/"
url2 = "https://fantasy.premierleague.com/api/entry/698498/history/"
url3 = "https://fantasy.premierleague.com/api/event/6/live/"
url4 = "https://fantasy.premierleague.com/api/fixtures"
url5 = 'https://fantasy.premierleague.com/api/element-summary/191/'

p4 = long_request(url4) 
d4 = json.loads(p4.text)
Fixtures = pd.DataFrame(d4)

p1 = long_request(url1)  

data1 = p1.text

d1 = json.loads(data1)
bigTable = pd.DataFrame(d1['elements'])
bigTable['full_name'] = bigTable['first_name'] + ' ' + bigTable['second_name']

teams = dict(zip(pd.DataFrame(d1['teams'])['id'],pd.DataFrame(d1['teams'])['name']))
players = dict(zip(bigTable['id'],bigTable['full_name']))
teamplayers = dict(zip(bigTable['id'],bigTable['team']))

Gameweeks = pd.DataFrame()
for i in range(1,2*len(teams) - 1):
    url = "https://fantasy.premierleague.com/api/event/" + str(i) + "/live/"
    p = long_request(url) 
    d = json.loads(p.text)
    nexTour = pd.DataFrame(d['elements'])

    if not nexTour.empty:
        nt1 = pd.DataFrame(nexTour['stats'].tolist())
        nt1['id'] = nexTour['id']
        nt1['gameweek'] = i
        nt1['fixture'] = ''
        for j in nexTour.index:
            if len(nexTour.at[j,'explain'])==1:
                nt1.at[j, 'fixture'] = nexTour.at[j,'explain'][0]['fixture']
            if len(nexTour.at[j,'explain'])==2:
                nt1.at[j, 'fixture'] = nexTour.at[j,'explain'][1]['fixture']
                newline = nt1.loc[j].copy()
                newline['fixture'] = nexTour.at[j,'explain'][0]['fixture']
                nt1 = nt1.append(newline, ignore_index=True)
            if len(nexTour.at[j,'explain'])==3:
                nt1.at[j, 'fixture'] = nexTour.at[j,'explain'][2]['fixture']
                newline1 = nt1.loc[j].copy()
                newline2 = nt1.loc[j].copy()
                newline1['fixture'] = nexTour.at[j,'explain'][1]['fixture']
                newline2['fixture'] = nexTour.at[j,'explain'][0]['fixture']
                nt1 = nt1.append(newline1, ignore_index=True)
                nt1 = nt1.append(newline2, ignore_index=True)
            if len(nexTour.at[j,'explain'])>3:
                print('Too many matches in Gameweek')
        Gameweeks = Gameweeks.append(nt1, ignore_index=True)
        print(i)

Gameweeks['team'] = [teamplayers[i] for i in Gameweeks['id']]

Gameweeks['threat'] = pd.to_numeric(Gameweeks['threat'])
Gameweeks['creativity'] = pd.to_numeric(Gameweeks['creativity'])

#Kill postponed matches lines

li =[]
for i in Gameweeks.index:
    if not is_finished(Gameweeks.at[i,'fixture']):
        li.append(i)
Gameweeks = Gameweeks.drop(li)

Gameweeks['team_a'] = [Fixtures[(pd.to_numeric(Fixtures['id']) == pd.to_numeric(Gameweeks.at[i,'fixture'])) & \
                                         ((Fixtures['team_a'] == Gameweeks.at[i,'team'])|\
                                          (Fixtures['team_h'] == Gameweeks.at[i,'team']))]['team_a'].values \
                            for i in Gameweeks.index]

Gameweeks['team_a'] = [int(Gameweeks.at[i,'team_a'][0]) if len(Gameweeks.at[i,'team_a'])==1 else '' for i in Gameweeks.index]

Gameweeks['team_h'] = [Fixtures[(pd.to_numeric(Fixtures['id']) == pd.to_numeric(Gameweeks.at[i,'fixture'])) & \
                                         ((Fixtures['team_a'] == Gameweeks.at[i,'team'])|\
                                          (Fixtures['team_h'] == Gameweeks.at[i,'team']))]['team_h'].values \
                            for i in Gameweeks.index]
Gameweeks['team_h'] = [int(Gameweeks.at[i,'team_h'][0]) if len(Gameweeks.at[i,'team_h'])==1 else '' for i in Gameweeks.index]

Gameweeks['teamAgainst'] = [Gameweeks.at[i,'team_a'] if Gameweeks.at[i,'team'] == Gameweeks.at[i,'team_h']\
                            else Gameweeks.at[i,'team_h'] \
                           for i in Gameweeks.index]
Gameweeks['side'] = ['home' if Gameweeks.at[i,'team'] == Gameweeks.at[i,'team_h']\
                            else 'away' \
                           for i in Gameweeks.index]

del Gameweeks['team_a']
del Gameweeks['team_h']
Gameweeks.to_csv(Path('in/fplgameweeks.csv'), index=False)

Gameweeks

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38


,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,in_dreamteam,influence,...,saves,threat,total_points,yellow_cards,id,gameweek,fixture,team,teamAgainst,side
0,0,1,27,1,0.0,0,0,1.8,False,18.0,...,2,0.0,7,0,14,1,10,1,13,away
1,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0.0,0,0,27,1,6,2,17,away
2,0,2,29,1,0.0,0,0,2.5,False,25.4,...,3,0.0,9,0,47,1,7,4,18,away
3,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0.0,0,0,48,1,7,4,18,away
4,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0.0,0,0,72,1,2,3,15,home
5,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0.0,0,0,73,1,2,3,15,home
6,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0.0,0,0,74,1,2,3,15,home
7,0,1,28,1,10.0,0,0,3.6,False,25.6,...,3,0.0,8,0,93,1,3,5,16,home
8,0,0,15,0,0.0,3,0,2.4,False,24.2,...,3,0.0,2,0,94,1,6,2,17,away
9,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0.0,0,0,95,1,3,5,16,home
